[Flow]
1. unify to xlsx file before inserting functions
2. insert functions (=GOOGLETRANSLATE, =DETECTLANGUAGE)
3. load the XLSX file into GDRIVE
4. Convert XLSX file to GSHEET in GDRIVE 
5. Download GSHEET as CSV file locally

[What you need]
1. a GOOGLE account
2. empty GDRIVE folder
3. GCP authentication json file key (enable GOOGLE DRIVE API and GOOGLE SPREAD SHEET API)

In [ ]:
#Convert CSV files to XLSX files
import pandas as pd
import glob
import os

folder_path = "PATH"
files = glob.glob(os.path.join(folder_path, "*.csv"))  
print(files)  

for file in files:
    try:
        df = pd.read_csv(file, index_col=None, header=0, on_bad_lines='skip')
        
        new_file_name = os.path.splitext(file)[0] + '.xlsx'
        
        df.to_excel(new_file_name, index=False, engine='openpyxl')  
        
        print(f"Converted {file} to {new_file_name}")
        
        os.remove(file)
        print(f"Deleted {file}")
        
    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}")
    except Exception as e:
        print(f"Unexpected error converting {file}: {e}")

In [ ]:
#Insert function (comment)
import os
from openpyxl import load_workbook

folder_path = "PATH"  

for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)
        
        wb = load_workbook(file_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            ws['F1'] = 'Translated Comment'  
            ws['G1'] = 'language'  
            
            for row in range(2, ws.max_row + 1):  
                # TRANSLATE
                ws[f'F{row}'] = f'=GOOGLETRANSLATE(B{row},"auto","en")'
                
                # DETECTLANGUAGE
                ws[f'G{row}'] = f'=DETECTLANGUAGE(B{row})'
                
        wb.save(file_path)

print("Complete!")

In [ ]:
#Insert function (subtitle)
import io
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Google API
SERVICE_ACCOUNT_FILE = 'PATH'

SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']

def authenticate_google_services():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    
    drive_service = build('drive', 'v3', credentials=creds)
    sheets_service = build('sheets', 'v4', credentials=creds)
    
    return drive_service, sheets_service

def list_xlsx_files(drive_service, folder_id):
    query = f"'{folder_id}' in parents and mimeType = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'"
    files = []
    page_token = None
    
    while True:
        response = drive_service.files().list(q=query, pageToken=page_token).execute()
        files.extend(response.get('files', []))
        
        page_token = response.get('nextPageToken')
        if not page_token:
            break
    
    print(f"Found {len(files)} files in folder {folder_id}")
    return files

def convert_to_spreadsheet(drive_service, file_id, file_name, new_folder_id):
    copied_file = drive_service.files().copy(
        fileId=file_id,
        body={
            'name': file_name,
            'mimeType': 'application/vnd.google-apps.spreadsheet',
            'parents': [new_folder_id]
        }).execute()
    
    return copied_file

def list_existing_spreadsheets(drive_service, folder_id):
    query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.spreadsheet'"
    results = drive_service.files().list(q=query).execute()
    return {file['name'].lower(): file['id'] for file in results.get('files', [])}

def main():
    drive_service, sheets_service = authenticate_google_services()
    
    source_folder_id = 'Folder ID to be read' 
    target_folder_id = 'Folder ID to output' 
    
    existing_spreadsheets = list_existing_spreadsheets(drive_service, target_folder_id)
    
    files = list_xlsx_files(drive_service, source_folder_id)

    for file in files:
        file_id = file['id']
        file_name = file['name']
        
        base_file_name = os.path.splitext(file_name)[0].lower()

        if base_file_name in existing_spreadsheets:
            print(f"Skipping file: {file_name} as it already exists.")
            continue
        
        try:
            print(f"Converting file: {file_name} (ID: {file_id})")
            converted_file = convert_to_spreadsheet(drive_service, file_id, file_name, target_folder_id)
            print(f"File converted and saved: {converted_file['name']} (ID: {converted_file['id']})")
        except Exception as e:
            print(f"Error converting file {file_name} (ID: {file_id}): {e}")
            continue

if __name__ == '__main__':
    main()

In [ ]:
#gsheet→csv
import io
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Google API
SERVICE_ACCOUNT_FILE = 'PATH'

SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']


def authenticate_google_services():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    
    drive_service = build('drive', 'v3', credentials=creds)
    sheets_service = build('sheets', 'v4', credentials=creds)
    
    return drive_service, sheets_service

def list_spreadsheet_files(drive_service, folder_id):
    query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.spreadsheet'"
    files = []
    page_token = None
    
    while True:
        response = drive_service.files().list(q=query, pageToken=page_token).execute()
        files.extend(response.get('files', []))
        
        page_token = response.get('nextPageToken')
        if not page_token:
            break
    
    print(f"Found {len(files)} spreadsheets in folder {folder_id}")
    return files

def download_spreadsheet_as_csv(drive_service, file_id, local_filename):
    request = drive_service.files().export_media(
        fileId=file_id,
        mimeType='text/csv'
    )
    
    with io.FileIO(local_filename, 'wb') as output_file:
        downloader = MediaIoBaseDownload(output_file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%.")
    
    print(f"CSV file saved as {local_filename}")

def main():
    drive_service, sheets_service = authenticate_google_services()
    
    source_folder_id = 'Folder ID to be read'  
    
    files = list_spreadsheet_files(drive_service, source_folder_id)

    for file in files:
        file_id = file['id']
        file_name = file['name']
        
        local_filename = os.path.join("PATH", f"{file_name}.csv")  # Specify destination file name
        
        try:
            print(f"Downloading file: {file_name} (ID: {file_id}) as CSV")
            download_spreadsheet_as_csv(drive_service, file_id, local_filename)
        except Exception as e:
            print(f"Error downloading file {file_name} (ID: {file_id}) as CSV: {e}")
            continue

if __name__ == '__main__':
    main()

In [ ]:
#　Consolidate csv files into one
import pandas as pd
import glob
import os

folder_path = "PATH"
files = glob.glob(folder_path + "/*.csv")
print(f"Found files: {files}")

lists = []
for file in files:
    try:
        filename = os.path.splitext(os.path.basename(file))[0]
        if os.path.getsize(file) > 0:
            df = pd.read_csv(file, index_col=None, header=0)
            df['Filename'] = filename
            lists.append(df)
            print(f"File read: {file}, Shape: {df.shape}")
        else:
            print(f"Skipped: {file}")
    except pd.errors.EmptyDataError:
        print(f"Error: {file} ")
    except Exception as e:
        print(f"Error: {file} : {e}")

if lists:  
    df = pd.concat(lists, axis=0, ignore_index=True)
    df.to_csv("/Users/naganawahisatoshi/Downloads/Logistics/Subtitle.csv", index=False, encoding='utf-8')
    print("Complete!")
else:
    print("No data.")

In [ ]:
#File count
import os

def count_files_in_directory(directory_path):
    file_count = 0
    for root, dirs, files in os.walk(directory_path):
        file_count += len(files)
    return file_count

directory_path = 'PATH' 
print(f'File count: {count_files_in_directory(directory_path)}')


In [ ]:
import csv

def count_rows_in_csv(file_path):
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        row_count = sum(1 for row in reader)  
    return row_count

file_path = "PATH"  
total_rows = count_rows_in_csv(file_path)
print(f"Number of rows in CSV file: {total_rows}")